In [ ]:
from dolfinx import mesh, fem, plot, io, default_scalar_type
import dolfinx.geometry
from mpi4py import MPI
import ufl
import numpy as np
from dolfinx.fem.petsc import LinearProblem


# Defining variables
L = 0.06
W = 0.05
H = 0.03
rho = 1
nu = 0.48
E = 0.12e6 
lambda_ = (E * nu) / ((1 + nu) * (1 - 2 * nu))
mu = E/(1 + 2 * nu)


# Defining the mesh
domain = mesh.create_box(MPI.COMM_WORLD, [np.array([0, 0, 0]), np.array([L, W, H])], # Defines the bounding box of the mesh
                         [24, 9, 16], cell_type=mesh.CellType.hexahedron) # Number of nodes in x, y, z and element shape
V = fem.functionspace(domain, ("Lagrange", 1, (domain.geometry.dim, ))) # Defines vector function space (element type, polynomial degree and dimension)

#Defining the boundary conditions
def clamped_boundary(x):
    return np.isclose(x[2], 0) # Defines the boundary, where z=0

fdim = domain.topology.dim - 1 # Dimensions of the boundary facets
boundary_facets = mesh.locate_entities_boundary(domain, fdim, clamped_boundary) # Defines boundary facets
u_D = np.array([0, 0, 0], dtype=default_scalar_type) # Defines the value of clamped nodes
bc = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, boundary_facets), V) # Applies the boundary condition

ds = ufl.Measure("ds", domain=domain) # Defining the integration measure

def epsilon(u): #strain
    return ufl.sym(ufl.grad(u))  # Equivalent to 0.5*(∇(u) + ∇(u^t))

def sigma(u):
    return lambda_ * ufl.nabla_div(u) * ufl.Identity(len(u)) + 2 * mu * epsilon(u) # 

u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

x = ufl.SpatialCoordinate(domain)
center = ufl.as_vector([0.015, 0.025, 0.03])
radius = 0.003
chi_surf = ufl.conditional(ufl.lt(ufl.inner(x - center, x - center), radius**2), 1, 0)

n = ufl.FacetNormal(domain)

pressure = 2e5  # N/m²
traction_form = ufl.dot(-pressure * n, v) * chi_surf * ufl.ds


LHS = ufl.inner(sigma(u), epsilon(v)) * ufl.dx
T = fem.Constant(domain, default_scalar_type((0, 0, 0))) # Traction equals zero
RHS = traction_form

# Compute solution
problem = LinearProblem(LHS, RHS, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

# Calculate von Mises stress
s_d = sigma(uh) - 1. / 3 * ufl.tr(sigma(uh)) * ufl.Identity(len(uh)) # Formula for deviatoric stress
von_Mises = ufl.sqrt(3. / 2 * ufl.inner(s_d, s_d))
V_von_mises = fem.functionspace(domain, ("DG", 0))
stress_expr = fem.Expression(von_Mises, V_von_mises.element.interpolation_points())
von_Mises_stress = fem.Function(V_von_mises)
von_Mises_stress.interpolate(stress_expr)

sigmazz=ufl.dot(ufl.dot(ufl.as_vector([0, 0, 1]), sigma(uh)),ufl.as_vector([0, 0, 1]))
sigmazz_expr = fem.Expression(sigmazz, V_von_mises.element.interpolation_points())
sigmazz_func = fem.Function(V_von_mises)
sigmazz_func.interpolate(sigmazz_expr)
sigmazz_func.name = "sigma_zz"

strainzz=ufl.dot(ufl.dot(ufl.as_vector([0, 0, 1]), epsilon(uh)),ufl.as_vector([0, 0, 1]))
strainzz_expr = fem.Expression(strainzz, V_von_mises.element.interpolation_points())
strainzz_func = fem.Function(V_von_mises)
strainzz_func.interpolate(strainzz_expr)
strainzz_func.name = "strain_zz"

strainxx=ufl.dot(ufl.dot(ufl.as_vector([1, 0, 0]), epsilon(uh)),ufl.as_vector([1, 0, 0]))
strainxx_expr = fem.Expression(strainxx, V_von_mises.element.interpolation_points())
strainxx_func = fem.Function(V_von_mises)
strainxx_func.interpolate(strainxx_expr)
strainxx_func.name = "strain_xx"

# Save to XDMF file for ParaView
with io.XDMFFile(domain.comm, "beam_deflection.xdmf", "w") as file:
    # Save mesh
    file.write_mesh(domain)
    
    # Save displacement field with a name
    uh.name = "Displacement"
    file.write_function(uh)
    
    # Save von Mises stress with a name
    von_Mises_stress.name = "von Mises stress"
    file.write_function(von_Mises_stress)

    file.write_function(sigmazz_func)
    file.write_function(strainzz_func)
    file.write_function(strainxx_func)
    
# Define the sensor points
sensor_points = [
    np.array([0.01, 0.025, 0]),
    np.array([0.02, 0.025, 0]),
    np.array([0.03, 0.025, 0]),
    np.array([0.04, 0.025, 0]),
    np.array([0.05, 0.025, 0]),
]
# Get all cell centroids
cell_centroids = np.zeros((domain.topology.index_map(domain.topology.dim).size_local, 3))
for cell_id in range(len(cell_centroids)):
    cell_geometry = domain.geometry.x[domain.topology.connectivity(domain.topology.dim, 0).links(cell_id)]
    cell_centroids[cell_id] = np.mean(cell_geometry, axis=0)

# For each sensor point, find the closest cell centroid
sensor_data = {}


for pt in sensor_points:
    distances = np.linalg.norm(cell_centroids - pt, axis=1)
    closest_cell = np.argmin(distances)

    vm_val = von_Mises_stress.x.array[closest_cell]
    sigzz_val = sigmazz_func.x.array[closest_cell]
    strainzz_val = strainzz_func.x.array[closest_cell]

    sensor_data[tuple(pt)] = {
    "von_mises": float(vm_val),
    "sigma_zz": float(sigzz_val),
    "strain_zz": float(strainzz_val)
    }

# Example print
for point, values in sensor_data.items():
    print(f"Point {point} → von Mises: {values['von_mises']:.3f}, σ_zz: {values['sigma_zz']:.3f}, ε_zz: {values['strain_zz']:.10f}")